# Find your ideal date here 

In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import random
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.filterwarnings("ignore", category=SettingWithCopyWarning)

In [2]:
data = pd.read_csv('okcupid_profiles.csv')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format
data.shape

(59946, 31)

In [3]:
data.head(10)

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,height,income,job,last_online,location,offspring,pets,religion,sign,smokes,speaks,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
0,22,single,m,straight,a little extra,strictly anything,socially,never,working on college/university,"asian, white",75.00,-1,transportation,2012-06-28-20-30,"south san francisco, california","doesn't have kids, but might want them",likes dogs and likes cats,agnosticism and very serious about it,gemini,sometimes,english,about me: i would love to think that i was so...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...","books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...
1,35,single,m,straight,average,mostly other,often,sometimes,working on space camp,white,70.00,80000,hospitality / travel,2012-06-29-21-41,"oakland, california","doesn't have kids, but might want them",likes dogs and likes cats,agnosticism but not too serious about it,cancer,no,"english (fluently), spanish (poorly), french (...",i am a chef: this is what that means. 1. i am ...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,NaN,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories. my b...,NaN,NaN,i am very open and will share just about anyth...,NaN
2,38,available,m,straight,thin,anything,socially,NaN,graduated from masters program,NaN,68.00,-1,NaN,2012-06-27-09-10,"san francisco, california",NaN,has cats,NaN,pisces but it doesn&rsquo;t matter,no,"english, french, c++","i'm not ashamed of much, but writing public te...","i make nerdy software for musicians, artists, ...",improvising in different contexts. alternating...,my large jaw and large glasses are the physica...,okay this is where the cultural matrix gets so...,movement conversation creation contemplation t...,NaN,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ..."
3,23,single,m,straight,thin,vegetarian,socially,NaN,working on college/university,white,71.00,20000,student,2012-06-28-14-22,"berkeley, california",doesn't want kids,likes cats,NaN,pisces,no,"english, german (poorly)",i work in a library and go to school. . .,reading things written by old dead people,playing synthesizers and organizing books acco...,socially awkward but i do my best,"bataille, celine, beckett. . . lynch, jarmusch...",NaN,cats and german philosophy,NaN,NaN,you feel so inclined.
4,29,single,m,straight,athletic,NaN,socially,never,graduated from college/university,"asian, black, other",66.00,-1,artistic / musical / writer,2012-06-27-21-26,"san francisco, california",NaN,likes dogs and likes cats,NaN,aquarius,no,english,hey how's it going? currently vague on the pro...,work work work work + play,creating imagery to look at: http://bagsbrown....,i smile a lot and my inquisitive nature,"music: bands, rappers, musicians at the moment...",NaN,NaN,NaN,NaN,NaN
5,29,single,m,straight,average,mostly anything,socially,NaN,graduated from college/university,white,67.00,-1,computer / hardware / software,2012-06-29-19-18,"san francisco, california","doesn't have kids, but might want them",likes cats,atheism,taurus,no,"english (fluently), chinese (okay)","i'm an australian living in san francisco, but...",building awesome stuff. figuring out what's im...,imagining random shit. laughing at aforementio...,i have a big smile. i also get asked if i'm we...,"books: to kill a mockingbird, lord of the ring...","like everyone else, i love my friends and fami...",what my contribution to the world is going to ...,out with my friends!,i c

In [4]:
data['name'] = data.apply(lambda row: random.choice(pd.read_csv('male.txt', header=None)[0]) if row['sex'] == 'm' else random.choice(pd.read_csv('female.txt', header=None)[0]), axis=1)

data['user_id'] = range(1, len(data) + 1)

data = pd.concat([data['user_id'], data.drop('user_id', axis=1)], axis=1)

data.head(100)

data2=data

In [5]:
data.columns

Index(['user_id', 'age', 'status', 'sex', 'orientation', 'body_type', 'diet',
       'drinks', 'drugs', 'education', 'ethnicity', 'height', 'income', 'job',
       'last_online', 'location', 'offspring', 'pets', 'religion', 'sign',
       'smokes', 'speaks', 'essay0', 'essay1', 'essay2', 'essay3', 'essay4',
       'essay5', 'essay6', 'essay7', 'essay8', 'essay9', 'name'],
      dtype='object')

# Text-preprocessing

In [6]:
data['merged_essays'] = data[['essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7', 'essay8', 'essay9', 'status', 'orientation', 'body_type', 'education', 'ethnicity', 'job', 'location']].apply(lambda x: ' '.join(map(str, x)), axis=1)

columns_to_keep = ['user_id','name', 'age',  'sex',  'height', 'income', 'merged_essays']

data = data[columns_to_keep]

data.head(10)

,user_id,name,age,sex,height,income,merged_essays
0,1,Zyaire,22,m,75.00,-1,about me: i would love to think that i was so...
1,2,Allan,35,m,70.00,80000,i am a chef: this is what that means. 1. i am ...
2,3,Clark,38,m,68.00,-1,"i'm not ashamed of much, but writing public te..."
3,4,Jamir,23,m,71.00,20000,i work in a library and go to school. . . read...
4,5,Samuel,29,m,66.00,-1,hey how's it going? currently vague on the pro...
5,6,Roman,29,m,67.00,-1,"i'm an australian living in san francisco, but..."
6,7,Edith,32,f,65.00,-1,life is about the little things. i love to lau...
7,8,Elowyn,31,f,65.00,-1,"nan writing. meeting new people, spending time..."
8,9,Molly,24,f,67.00,-1,"nan oh goodness. at the moment i have 4 jobs, ..."
9,10,Lucian,37,m,65.00,-1,my names jake. i'm a creative guy and i look f...


In [7]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

data['merged_essays'] = data['merged_essays'].apply(preprocess_text)

In [8]:
data.head(10)

,user_id,name,age,sex,height,income,merged_essays
0,1,Zyaire,22,m,75.00,-1,would love think kind intellectual either dumb...
1,2,Allan,35,m,70.00,80000,chef mean 1. workaholic 2. love cook regardles...
2,3,Clark,38,m,68.00,-1,'m ashamed much writing public text online dat...
3,4,Jamir,23,m,71.00,20000,work library go school reading thing written o...
4,5,Samuel,29,m,66.00,-1,hey 's going currently vague profile know come...
5,6,Roman,29,m,67.00,-1,'m australian living san francisco n't hold sp...
6,7,Edith,32,f,65.00,-1,life little thing love laugh 's easy one find ...
7,8,Elowyn,31,f,65.00,-1,nan writing meeting new people spending time f...
8,9,Molly,24,f,67.00,-1,nan oh goodness moment 4 job 'd nice find one ...
9,10,Lucian,37,m,65.00,-1,name jake 'm creative guy look others 'm easy ...


# Missing Value Imputing

In [9]:
missing_values_count = data.isna().sum()
missing_values_count

user_id          0
name             0
age              0
sex              0
height           3
income           0
merged_essays    0
dtype: int64

In [10]:
for column in missing_values_count.index:
    if missing_values_count[column] > 0:
        random_values = data[column].dropna().sample(missing_values_count[column], replace=True)
        data.loc[data[column].isna(), column] = random_values.values

data.head(10)

,user_id,name,age,sex,height,income,merged_essays
0,1,Zyaire,22,m,75.00,-1,would love think kind intellectual either dumb...
1,2,Allan,35,m,70.00,80000,chef mean 1. workaholic 2. love cook regardles...
2,3,Clark,38,m,68.00,-1,'m ashamed much writing public text online dat...
3,4,Jamir,23,m,71.00,20000,work library go school reading thing written o...
4,5,Samuel,29,m,66.00,-1,hey 's going currently vague profile know come...
5,6,Roman,29,m,67.00,-1,'m australian living san francisco n't hold sp...
6,7,Edith,32,f,65.00,-1,life little thing love laugh 's easy one find ...
7,8,Elowyn,31,f,65.00,-1,nan writing meeting new people spending time f...
8,9,Molly,24,f,67.00,-1,nan oh goodness moment 4 job 'd nice find one ...
9,10,Lucian,37,m,65.00,-1,name jake 'm creative guy look others 'm easy ...


In [11]:
missing_values_count = data.isna().sum()
missing_values_count

user_id          0
name             0
age              0
sex              0
height           0
income           0
merged_essays    0
dtype: int64

# Text-vectorization

In [12]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens

data['tokenized_essays'] = data['merged_essays'].apply(preprocess_text)

word2vec_model = Word2Vec(sentences=data['tokenized_essays'], vector_size=100, window=5, min_count=1, workers=4)

def get_vector(tokens):
    if len(tokens) > 0:
        return np.mean(word2vec_model.wv[tokens], axis=0)
    else:
        return np.zeros(100)

data['merged_essays_vector'] = data['tokenized_essays'].apply(get_vector)

columns_to_remove = ['merged_essays', 'tokenized_essays']
data = data.drop(columns=columns_to_remove)

data.head(10)

,user_id,name,age,sex,height,income,merged_essays_vector
0,1,Zyaire,22,m,75.00,-1,"[0.0417684, -0.1888449, -0.11609026, 0.0352630..."
1,2,Allan,35,m,70.00,80000,"[0.10676709, -0.13992967, -0.50543684, -0.0094..."
2,3,Clark,38,m,68.00,-1,"[0.15741321, -0.10107984, -0.055960935, -0.178..."
3,4,Jamir,23,m,71.00,20000,"[0.119092606, -0.10122968, -0.029807512, 0.009..."
4,5,Samuel,29,m,66.00,-1,"[0.23207659, -0.16339284, -0.4114032, -0.16676..."
5,6,Roman,29,m,67.00,-1,"[-0.003428431, -0.447545, 0.10663356, -0.17463..."
6,7,Edith,32,f,65.00,-1,"[0.1719349, 0.051535178, 0.018730834, -0.66310..."
7,8,Elowyn,31,f,65.00,-1,"[-0.0693251, 0.42611042, 0.068054795, -0.25110..."
8,9,Molly,24,f,67.00,-1,"[0.16103368, -0.30178335, 0.022007156, -0.0023..."
9,10,Lucian,37,m,65.00,-1,"[0.15308046, -0.24305363, -0.16275188, -0.4216..."


In [13]:
data.columns

Index(['user_id', 'name', 'age', 'sex', 'height', 'income',
       'merged_essays_vector'],
      dtype='object')

# Scaling the dataset

In [14]:
from sklearn.preprocessing import MinMaxScaler

columns_to_scale = ['age', 'height', 'income']

scaler = MinMaxScaler()

data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

data.head(10)

,user_id,name,age,sex,height,income,merged_essays_vector
0,1,Zyaire,0.04,m,0.79,0.00,"[0.0417684, -0.1888449, -0.11609026, 0.0352630..."
1,2,Allan,0.18,m,0.73,0.08,"[0.10676709, -0.13992967, -0.50543684, -0.0094..."
2,3,Clark,0.22,m,0.71,0.00,"[0.15741321, -0.10107984, -0.055960935, -0.178..."
3,4,Jamir,0.05,m,0.74,0.02,"[0.119092606, -0.10122968, -0.029807512, 0.009..."
4,5,Samuel,0.12,m,0.69,0.00,"[0.23207659, -0.16339284, -0.4114032, -0.16676..."
5,6,Roman,0.12,m,0.70,0.00,"[-0.003428431, -0.447545, 0.10663356, -0.17463..."
6,7,Edith,0.15,f,0.68,0.00,"[0.1719349, 0.051535178, 0.018730834, -0.66310..."
7,8,Elowyn,0.14,f,0.68,0.00,"[-0.0693251, 0.42611042, 0.068054795, -0.25110..."
8,9,Molly,0.07,f,0.70,0.00,"[0.16103368, -0.30178335, 0.022007156, -0.0023..."
9,10,Lucian,0.21,m,0.68,0.00,"[0.15308046, -0.24305363, -0.16275188, -0.4216..."


# Filtering using Cosine-Similarity

In [15]:
user_id = 49
user_id2=user_id
age_up = 5
age_down = 5
n=3

user_data = data2.loc[data2['user_id'] == user_id, ['age', 'sex']].iloc[0]
current_user_age = user_data['age']
current_user_sex = user_data['sex']

print(f"\nSex of the Current User (user_id {user_id}): {current_user_sex}")
print(f"Age of the Current User (user_id {user_id}): {current_user_age}\n")

filtered_users = data2[(data2['age'] >= current_user_age - age_down) & (data2['age'] <= current_user_age + age_up) & (data2['sex'] != current_user_sex)]




Sex of the Current User (user_id 49): f
Age of the Current User (user_id 49): 31



In [16]:
user_id = user_id2
user_id

49

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

user_vector = data.loc[data['user_id'] == user_id, 'merged_essays_vector'].values[0]

similarities = []
similar_users = []

for index, row in filtered_users.iterrows():
    current_user_id = row['user_id']
    current_user_vector = data.loc[data['user_id'] == current_user_id, 'merged_essays_vector'].values[0]
    
    similarity = cosine_similarity(user_vector.reshape(1, -1), current_user_vector.reshape(1, -1))[0, 0]
    
    similarities.append(similarity)
    similar_users.append(current_user_id)


In [18]:
user_id =user_id2
user_id

49

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

user_vector = data.loc[data['user_id'] == user_id, 'merged_essays_vector'].values[0]

similarities = []
similar_users = []

for index, row in filtered_users.iterrows():
    current_user_id = row['user_id']
    current_user_vector = data.loc[data['user_id'] == current_user_id, 'merged_essays_vector'].values[0]
    
    similarity = cosine_similarity(user_vector.reshape(1, -1), current_user_vector.reshape(1, -1))[0, 0]
    
    similarities.append(similarity)
    similar_users.append(current_user_id)

top_indices = np.argsort(similarities)[-n:][::-1]

last_filtered = [similar_users[i] for i in top_indices]

print(f"\nTop {n} Users with Maximum Cosine Similarity:")
for user_id in last_filtered:
    print(f"User ID: {user_id}")



Top 3 Users with Maximum Cosine Similarity:
User ID: 24233
User ID: 20340
User ID: 845


In [20]:
user_id = user_id2
user_id

49

In [21]:
pd.set_option('display.max_rows', 1000000)
pd.set_option('display.max_columns', 1000000)

# Showing best results according to the userid and conditions mentioned

In [22]:
columns_to_display = ['name', 'age', 'status', 'sex', 'orientation', 'body_type', 'diet',
                       'drinks', 'drugs', 'education', 'ethnicity', 'height', 'income', 'job',
                       'location', 'offspring', 'pets', 'religion', 'sign', 'smokes', 'speaks']

print(f"\nOriginal User ID: {user_id}")
original_user_data = data2[data2['user_id'] == user_id][columns_to_display]
for col in columns_to_display:
    print(f"{col}: {original_user_data.iloc[0][col]}")
print("\n" + "-"*120 + "\n")

for choice, user_id in enumerate(last_filtered, 1):
    print(f"\nChoice {choice}")
    user_data = data2[data2['user_id'] == user_id][columns_to_display]

    for col in columns_to_display:
        print(f"{col}: {user_data.iloc[0][col]}")

    print("\n" + "-"*120 + "\n")



Original User ID: 49
name: Bexley
age: 31
status: single
sex: f
orientation: bisexual
body_type: thin
diet: strictly anything
drinks: socially
drugs: sometimes
education: dropped out of space camp
ethnicity: nan
height: 67.0
income: 1000000
job: artistic / musical / writer
location: san francisco, california
offspring: doesn't have kids
pets: nan
religion: nan
sign: nan
smokes: no
speaks: english (fluently), belarusan, gujarati, lisp, russian (fluently)

------------------------------------------------------------------------------------------------------------------------


Choice 1
name: Walter
age: 36
status: seeing someone
sex: m
orientation: bisexual
body_type: fit
diet: anything
drinks: socially
drugs: sometimes
education: graduated from college/university
ethnicity: nan
height: 70.0
income: -1
job: artistic / musical / writer
location: san francisco, california
offspring: doesn't have kids
pets: likes dogs and likes cats
religion: agnosticism and laughing about it
sign: scorpio